# MyFitnessPal Diary Parser

In [1]:
# Libraries
from bs4 import BeautifulSoup
from pprint import pprint
from datetime import datetime
import pandas as pd
import pickle
import logging

from configparser import ConfigParser
import os

# Notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)

# Logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO) # DEBUG

In [2]:
%%capture
# Parse parameters in config.ini
config = ConfigParser()
configfilePath = (os.path.join(os.getcwd(), 'config.ini'))
config.read(configfilePath)

input_dir = config.get('GLOBAL', 'input_folder')
input_file = config.get('GLOBAL', 'input_file')
data_file = input_dir + input_file
output_dir = config.get('GLOBAL', 'output_folder')
output_file = config.get('GLOBAL', 'output_file')

# Functions

In [3]:
def get_report_meta(html):
    """
    Extract metadata from a MyFitnessPal diary, including:
    * Date Range
    * Food statistic names (e.g. [Calories, Carbs, Protein, ...])
    * Exercise statistic names (e.g. [Calories, Minutes, Sets, ...])
      
    Args:
        html (BeautifulSoup): MyFitnessPal diary 
        
    Returns:
        dataframe
    """
    logging.debug('get_report_meta ...')
    result_dict = {}
    
    # Grab date range
    # "<input ... id="from" ... value="2017-11-01">
    result_dict['Date From'] = datetime.strptime(\
                [x["value"] for x in html.findAll("input", {"id": "from"})][0], \
                '%Y-%m-%d').date()
    result_dict['Date To'] = datetime.strptime(\
                [x["value"] for x in html.findAll("input", {"id": "to"})][0], \
                '%Y-%m-%d').date()
  
    
    # Extract dataset types (e.g. 'Foods' and 'Exercises').
    # Types are declared in the first <td> column of each <thead> section.
    # "<thead><tr><td class="first">Foods</td><td>Calories</td><td>...</td></tr></thead>"
    table_type_list = []   
    table_dict_list = []
    
    for thead in html.find_all('thead'):
        
        # Get column headers from this <thead>
        columns = thead.find('tr').findChildren('td')

        # First column = Data set type ('Foods' or 'Exercises')
        table_type = columns[0].get_text()  

        # Record unique data set types and their associated statistic types 
        if not table_type in table_type_list:

            table_type_list.append(table_type)

            # Remaining table columns headers statistic names (e.g. 'Carbs', 'Minutes).
            # List them.
            this_table_stats = {}
            this_table_stats['Stats List'] = [x.get_text() for x in columns[1:]]
            result_dict[table_type] = this_table_stats
              
    return pd.DataFrame(result_dict)
    

In [4]:
def get_daily_logs(html, totals_only=True):
    """
    Parse html into embedded dictionaries. 
    
    Log dates are tagged with <h2>, which are followed by that date's log entries.
    
    'Foods' and 'Exercises' log entries are each tagged by <table>. The tables
    contain individual food items and exercises.
    
    'Food Notes' and 'Exercise Notes' are tagged with <h4>.
    
    For example, the html below will result in 2 embedded dictionaries from 2-days of logging.
        <h2 id="date">August 25, 2018</h2>
        <table id="food">...</table>
        <table id="exercises">...</table>
        <h2 id="date">August 26, 2018</h2>
        <table id="food">...</table>
   
    Args:
        html (BeautifulSoup): MyFitnessPal diary 
        totals_only (boolean): return daily statistic totals if True, 
                               otherwise include log entry details
        
    Returns:
        dictionary (embedded)
    """    

    logging.debug('get_daily_logs ...')
    result_dict = {x:[] for x in ['Foods', 'Exercises', 'Notes']}


    # Get 1st <h2> (log date), then find all its siblings.
    # Siblings of interest are <table> (log entires) and <h2> (the next log date).
    h2 = html.find('h2', {'id':'date'})
    h2_date = h2.get_text()  

    for tag in html.find('h2').next_siblings:

        # If tag is <table> then the date is unchanged,
        # so parse the table into the day's dictionary.
        if tag.name == 'table':
            this_table_dict, this_table_type = table_to_dict(tag, totals_only)
            this_table_dict['Date'] = h2_date
            
            result_dict[this_table_type].append(this_table_dict)   
        
        # If tag is <h4> it's either 'Food Notes' or 'Exercise Notes'.
        # Add notes to dictionary.
        elif tag.name == 'h4':
            h4_notetype = tag.get_text()
            note_text = tag.find_next_sibling('p').get_text()
            this_note_dict = {'Date': h2_date,
                              'Note Type': h4_notetype,
                              'Note': note_text
                             }
            #this_note_dict['Date'] = h2_date
            #this_note_dict['Note Type'] = h4_notetype
            #this_note_dict['Note'] = note_text
            result_dict['Notes'].append(this_note_dict)
            
        # If tag is <h2> then we're on to the next date. 
        elif tag.name == 'h2':
            h2_date = tag.get_text()  

        else:
            pass
        
    return result_dict


In [5]:
def table_to_dict(html, totals_only=True):
    '''
    Convert HTML <table> data into a dictionary.
    
    Each <table> body has 3 sections: 
        * <thead>, contains the log type ('Foods' or 'Exercises').
        * <tbody>, contains multiple datasets ('Breakfast' etc, or 'Cardiovascular' etc), 
                   as well as each item in the dataset(s) (ingredients, exercises) and stats.
        * <tfoot>, contains the total statistics for the log entry dataset.
    
    Args: 
        html (BeautifulSoup): HTML <table> body
        totals_only (boolean): return daily statistic totals if True, 
                               otherwise include log entry details
        
    Returns:
        dictionary: a single dated log entry
        string: 'Foods' or 'Exercises'
    '''
    logging.debug('table_to_dict ...')
    result_log_type = ''
    result_dict = {}  

    # Get column headers from <thead>'s <td> tags.
    #     <thead> <tr> <td>Foods</td> <td>Calories</td> <td>Carbs</td> ... </tr> </thead>
    columns = html.find('thead').find('tr').findChildren('td')
    
    # Get log type ('Foods' or 'Exercises') from the first column header.
    result_log_type = columns[0].get_text()  
    #result_dict['Set Type'] = result_log_type
    
    # Get the dataset type ('Breakfast' etc, or 'Cardiovascular' etc), along with all its items.
    if not totals_only:
        tbody = html.find('tbody')

        # Get the first dataset type from the first row. 
        #    <tbody> <tr> <td>Breakfast</td> </tr> ...
        first_row = tbody.find('tr', {'class': 'title'})
        logging.debug(f'first_row = {first_row}')
        this_dataset_type = first_row.findChild('td').get_text()

        # Then get the rest of the rows in <tbody>, which will be either a dataset item and its details ... 
        #    <tr> <td>'Oil, avocado, 1 tbsp</td><td>124</td><td>0g</td>... </tr>  
        # or the next dataset type followed by its items.
        table_rows = first_row.find_next_siblings('tr')
        

        this_item_dict = {}  # dictionary of an item (e.g. 'Oil, avocado')
        this_item_list = []  # list of item dictionaries 

         
        for row in table_rows:
            # If a row's tag (<tr>) has a class, then it's a new dataset type.
            #     Append the previous dataset's item list to the result_dict, 
            #     then reset the dataset_type to this new one.
            if row.has_attr('class'):  
                result_dict[this_dataset_type] = this_item_list 
                this_dataset_type = row.findChild('td').get_text()

            # Else this row is an item, so parse and add it to the current dataset's 
            # list of items.
            else:           
                # The first column contains the item name.
                item = row.findChild('td')
                item_text = item.get_text()

                # 'item' contains both it's name and amount ('Oil, avocado, 1 tbsp').
                this_item_dict['Item Name'] = ','.join(item_text.split(',')[:-1])
                this_item_dict['Amount'] = item_text.split(',')[-1:][0].strip()

                # The rest of the columns contain the item stats. 
                stats = item.find_next_siblings('td')
                stats_list = []
                for stat in stats:
                    stats_list.append(stat.get_text())
                this_item_dict['Stats'] = stats_list         

                this_item_list.append(this_item_dict)
                this_item_dict = {}

            # log the final dataset's items
            result_dict[this_dataset_type] = this_item_list 
        
        
    # Get summary totals from <tfoot>. Skip the first column, which is just 'TOTAL:'.
    table_totals = html.find('tfoot').find('tr').findChildren('td')
    result_dict['Totals'] = [x.get_text() for x in table_totals[1:]]
    
    return result_dict, result_log_type


In [6]:
def flatten_logs(log_list, dataset_type='Foods'):
    '''
    Flatten a list of daily logs (dictionaries) into a dataframe format.   
    
    `get_daily_logs` extracts daily log entries into a list of dictionaries 
    (each daily log entry a dictionary). Flatten each entry in the journal 
    (e.g. a food item or an exercise) is a row in the result dataframe.
    
    Args: 
        log_list (list): list embedded with dictionaries
        dataset_type (str): 'Foods' or 'Exercises'
        
    Returns:
        dataframe: a row for each log entry 
    '''
    
    logging.debug('flatten_logs ...')
    # Get stats columns for this dataset_type
    stats_list = report_meta_df[dataset_type]['Stats List']
    # Initialize result dataframe 
    result_df = pd.DataFrame(columns=['Date','Entry','Item','Amount'] + stats_list)
    #r_df = result_df.copy()
    notlist = ['Date','Totals']
    for log in log_list:
        # For all columns except 'Date' and 'Totals', flatten 
        # each column value (they are dictionaries)
        for key in [x for x in log.keys() if x not in notlist]:
            for entry in log[key]:
                """
                r_df = result_df.append({'Date': log['Date'],
                                              'Entry': key,
                                              'Item': entry['Item Name'],
                                              'Amount': entry['Amount']}, 
                                             ignore_index=True)
                """
                entry_dict = {'Date': log['Date'],
                              'Entry': key,
                              'Item': entry['Item Name'],
                              'Amount': entry['Amount']}
                stats_dict = dict(zip(stats_list, entry['Stats']))
                combined_dict = dict(entry_dict, **stats_dict)
                result_df = result_df.append(combined_dict, 
                                             ignore_index=True)
                
                
                
    return result_df

# Main

In [7]:
if __name__ == '__main__':
    
    logging.debug('main ...')  
    
    # Load soup
    with open(data_file, 'r') as f:
        page = f.read()
        
    logging.debug('Get soup ...')
    soup = BeautifulSoup(page, 'html.parser')
    
    
    # Get report meta data
    report_meta_df = get_report_meta(soup)
    

    # Get daily log entries for 'Foods' and 'Exercises'
    daily_log_dict = get_daily_logs(soup, totals_only=False)
    
    
    # Extract daily log into dataframes
    if len(daily_log_dict['Foods']) > 0:
        logging.debug('Extract food_log_df ...')
        food_log_df = pd.DataFrame(daily_log_dict['Foods']).set_index(['Date'])
    if len(daily_log_dict['Exercises']) > 0:
        logging.debug('Extract exercise_log_df ...')
        exercise_log_df = pd.DataFrame(daily_log_dict['Exercises']).set_index(['Date'])
    if len(daily_log_dict['Notes']) > 0:
        logging.debug('Extract notes_log_df ...')
        notes_log_df = pd.DataFrame(daily_log_dict['Notes']).set_index(['Date'])
    
    # Flatten the log dataframes (no need to flatten Notes log)
    logging.debug('Flatten exercise log ...')
    exercise_log_flat = flatten_logs(daily_log_dict['Exercises'], 'Exercises')
    logging.debug('Flatten food log ...')
    food_log_flat = flatten_logs(daily_log_dict['Foods'], 'Foods')

# Explore our new data structures

In [8]:
report_meta_df.head()

,Date From,Date To,Foods,Exercises
Stats List,2019-01-01,2019-01-14,"[Calories, Carbs, Fat, Protein, Cholest, Sodium, Sugars, Fiber]","[Calories, Minutes, Sets, Reps, Weight]"


In [9]:
daily_log_dict['Exercises']

[{'Cardiovascular': [{'Amount': 'walking dog',
    'Item Name': 'Walking, 3.0 mph, mod. pace',
    'Stats': ['124', '20', '\xa0']}],
  'Date': 'January 1, 2019',
  'Totals': ['124', '20', '0', '0', '0']},
 {'Date': 'January 2, 2019',
  'Strength Training': [{'Amount': 'Bent Arm Barbell Pullover',
    'Item Name': '',
    'Stats': ['\xa0', '3', '12', '\n']},
   {'Amount': 'Leg Curls',
    'Item Name': '',
    'Stats': ['\xa0', '3', '12', '\n']},
   {'Amount': 'Leg Extension',
    'Item Name': '',
    'Stats': ['\xa0', '3', '12', '\n']}],
  'Totals': ['0', '0', '9', '36', '0']},
 {'Cardiovascular': [{'Amount': 'walking dog',
    'Item Name': 'Walking, 3.0 mph, mod. pace',
    'Stats': ['124', '20', '\xa0']},
   {'Amount': 'light/moderate effort',
    'Item Name': 'Swimming laps, freestyle',
    'Stats': ['394', '30', '\xa0']}],
  'Date': 'January 3, 2019',
  'Totals': ['518', '50', '0', '0', '0']},
 {'Date': 'January 4, 2019',
  'Strength Training': [{'Amount': 'Bent Arm Barbell Pullover

In [10]:
food_log_df.head(2)

,Breakfast,Dinner,Lunch,Snacks,Totals
Date,,,,,
"January 1, 2019","[{'Item Name': 'Baked Cod - Baked Cod', 'Amount': '4.8 oz', 'Stats': ['166', '1g', '5g', '28g', '59mg', '552mg', '0g', '0g']}, {'Item Name': 'Sweet Potato Soup', 'Amount': '2 serving(s)', 'Stats': ['638', '103g', '15g', '32g', '52mg', '3,436mg', '40g', '17g']}, {'Item Name': 'Thompson Raisins - Raisin', 'Amount': '1 c', 'Stats': ['400', '100g', '0g', '4g', '0mg', '40mg', '92g', '8g']}, {'Item Name': 'Seeds, sunflower seed kernels, dry roasted, with salt added', 'Amount': '1 cup', 'Stats': ['745', '31g', '64g', '25g', '0mg', '838mg', '3g', '12g']}]","[{'Item Name': 'Baked Cod - Baked Cod', 'Amount': '4.8 oz', 'Stats': ['166', '1g', '5g', '28g', '59mg', '552mg', '0g', '0g']}, {'Item Name': 'Sweet Potato Soup', 'Amount': '2 serving(s)', 'Stats': ['638', '103g', '15g', '32g', '52mg', '3,436mg', '40g', '17g']}, {'Item Name': 'Thompson Raisins - Raisin', 'Amount': '1 c', 'Stats': ['400', '100g', '0g', '4g', '0mg', '40mg', '92g', '8g']}, {'Item Name': 'Seeds, sunflower seed kernels, dry roasted, with salt added', 'Amount': '1 cup', 'Stats': ['745', '31g', '64g', '25g', '0mg', '838mg', '3g', '12g']}]",NaN,NaN,"[1,949, 235g, 84g, 89g, 111mg, 4,866mg, 135g, 37g]"
"January 2, 2019","[{'Item Name': 'Chicken - Chicken Breast Boiled Without Skin', 'Amount': '1 ounce', 'Stats': ['56', '0g', '2g', '8g', '24mg', '20mg', '0g', '0g']}, {'Item Name': 'Organics roasted cashews - Cashews', 'Amount': '1 oz', 'Stats': ['160', '9g', '13g', '4g', '0mg', '95mg', '1g', '1g']}, {'Item Name': 'Thai Coconut Curry Soup', 'Amount': '1 serving(s)', 'Stats': ['375', '18g', '31g', '6g', '0mg', '2,274mg', '9g', '3g']}, {'Item Name': 'Sweet Potato - Roasted Slices', 'Amount': '0.75 lb(s)', 'Stats': ['300', '72g', '0g', '12g', '0mg', '120mg', '24g', '12g']}, {'Item Name': 'Angus Beef - Hamburger Patty 1/3 lb', 'Amount': '1 patty 151g.', 'Stats': ['320', '0g', '30g', '26g', '105mg', '105mg', '--g', '--g']}, {'Item Name': 'thrive life - freeze dried kale', 'Amount': '1.13 C', 'Stats': ['11', '2g', '0g', '1g', '0mg', '8mg', '0g', '2g']}, {'Item Name': 'Oil, avocado', 'Amount': '1 tbsp', 'Stats': ['124', '0g', '14g', '0g', '0mg', '0mg', '0g', '0g']}, {'Item Name': 'Onions, raw, 0.5 cup', 'Amount': 'chopped', 'Stats': ['32', '7g', '0g', '1g', '0mg', '3mg', '3g', '1g']}, {'Item Name': 'Organics roasted cashews - Cashews', 'Amount': '1 oz', 'Stats': ['160', '9g', '13g', '4g', '0mg', '95mg', '1g', '1g']}, {'Item Name': 'Organics roasted cashews - Cashews', 'Amount': '1 oz', 'Stats': ['160', '9g', '13g', '4g', '0mg', '95mg', '1g', '1g']}]","[{'Item Name': 'Chicken - Chicken Breast Boiled Without Skin', 'Amount': '1 ounce', 'Stats': ['56', '0g', '2g', '8g', '24mg', '20mg', '0g', '0g']}, {'Item Name': 'Organics roasted cashews - Cashews', 'Amount': '1 oz', 'Stats': ['160', '9g', '13g', '4g', '0mg', '95mg', '1g', '1g']}, {'Item Name': 'Thai Coconut Curry Soup', 'Amount': '1 serving(s)', 'Stats': ['375', '18g', '31g', '6g', '0mg', '2,274mg', '9g', '3g']}, {'Item Name': 'Sweet Potato - Roasted Slices', 'Amount': '0.75 lb(s)', 'Stats': ['300', '72g', '0g', '12g', '0mg', '120mg', '24g', '12g']}, {'Item Name': 'Angus Beef - Hamburger Patty 1/3 lb', 'Amount': '1 patty 151g.', 'Stats': ['320', '0g', '30g', '26g', '105mg', '105mg', '--g', '--g']}, {'Item Name': 'thrive life - freeze dried kale', 'Amount': '1.13 C', 'Stats': ['11', '2g', '0g', '1g', '0mg', '8mg', '0g', '2g']}, {'Item Name': 'Oil, avocado', 'Amount': '1 tbsp', 'Stats': ['124', '0g', '14g', '0g', '0mg', '0mg', '0g', '0g']}, {'Item Name': 'Onions, raw, 0.5 cup', 'Amount': 'chopped', 'Stats': ['32', '7g', '0g', '1g', '0mg', '3mg', '3g', '1g']}, {'Item Name': 'Organics roasted cashews - Cashews', 'Amount': '1 oz', 'Stats': ['160', '9g', '13g', '4g', '0mg', '95mg', '1g', '1g']}, {'Item Name': 'Organics roasted cashews - Cashews', 'Amount': '1 oz', 'Stats': ['160', '9g', '13g', '4g', '0mg', '95mg', '1g', '1g']}]",NaN,"[{'Item Name': 'Chicken - Chicken Breast B

In [11]:
try: 
    exercise_log_df.head()
except:
    pass

,Cardiovascular,Strength Training,Totals
Date,,,
"January 1, 2019","[{'Item Name': 'Walking, 3.0 mph, mod. pace', 'Amount': 'walking dog', 'Stats': ['124', '20', ' ']}]",NaN,"[124, 20, 0, 0, 0]"
"January 2, 2019",NaN,"[{'Item Name': '', 'Amount': 'Bent Arm Barbell Pullover', 'Stats': [' ', '3', '12', ' ']}, {'Item Name': '', 'Amount': 'Leg Curls', 'Stats': [' ', '3', '12', ' ']}, {'Item Name': '', 'Amount': 'Leg Extension', 'Stats': [' ', '3', '12', ' ']}]","[0, 0, 9, 36, 0]"
"January 3, 2019","[{'Item Name': 'Walking, 3.0 mph, mod. pace', 'Amount': 'walking dog', 'Stats': ['124', '20', ' ']}, {'Item Name': 'Swimming laps, freestyle', 'Amount': 'light/moderate effort', 'Stats': ['394', '30', ' ']}]",NaN,"[518, 50, 0, 0, 0]"
"January 4, 2019",NaN,"[{'Item Name': '', 'Amount': 'Bent Arm Barbell Pullover', 'Stats': [' ', '3', '12', ' ']}, {'Item Name': '', 'Amount': 'Leg Curls', 'Stats': [' ', '3', '12', ' ']}, {'Item Name': '', 'Amount': 'Leg Extension', 'Stats': [' ', '3', '12', ' ']}]","[0, 0, 9, 36, 0]"
"January 5, 2019","[{'Item Name': 'Walking, 3.0 mph, mod. pace', 'Amount': 'walking dog', 'Stats': ['124', '20', ' ']}, {'Item Name': 'Swimming laps, freestyle', 'Amount': 'light/moderate effort', 'Stats': ['394', '30', ' ']}]",NaN,"[518, 50, 0, 0, 0]"


In [12]:
try: 
    notes_log_df
except:
    pass

,Note,Note Type
Date,,
"January 1, 2019",Starting Whole30 today.,Food Notes
"January 1, 2019","Also, did a lot of hauling today.",Exercise notes
"January 8, 2019",Symptom - Knee Pain,Exercise notes
"January 11, 2019",Symptom - Knee Pain,Exercise notes


In [13]:
try: 
    exercise_log_flat.head()
except:
    pass

,Date,Entry,Item,Amount,Calories,Minutes,Sets,Reps,Weight
0,"January 1, 2019",Cardiovascular,"Walking, 3.0 mph, mod. pace",walking dog,124,20,,NaN,NaN
1,"January 2, 2019",Strength Training,,Bent Arm Barbell Pullover,,3,12,\n,NaN
2,"January 2, 2019",Strength Training,,Leg Curls,,3,12,\n,NaN
3,"January 2, 2019",Strength Training,,Leg Extension,,3,12,\n,NaN
4,"January 3, 2019",Cardiovascular,"Walking, 3.0 mph, mod. pace",walking dog,124,20,,NaN,NaN


In [14]:
dates = food_log_flat['Date'].unique().tolist()
print(f'\n{len(dates)} days logged between {dates[0]} and {dates[-1]}:')
dates
food_log_flat.head()
food_log_flat.tail()


14 days logged between January 1, 2019 and January 14, 2019:


['January 1, 2019',
 'January 2, 2019',
 'January 3, 2019',
 'January 4, 2019',
 'January 5, 2019',
 'January 6, 2019',
 'January 7, 2019',
 'January 8, 2019',
 'January 9, 2019',
 'January 10, 2019',
 'January 11, 2019',
 'January 12, 2019',
 'January 13, 2019',
 'January 14, 2019']

,Date,Entry,Item,Amount,Calories,Carbs,Fat,Protein,Cholest,Sodium,Sugars,Fiber
0,"January 1, 2019",Breakfast,Baked Cod - Baked Cod,4.8 oz,166,1g,5g,28g,59mg,552mg,0g,0g
1,"January 1, 2019",Breakfast,Sweet Potato Soup,2 serving(s),638,103g,15g,32g,52mg,"3,436mg",40g,17g
2,"January 1, 2019",Breakfast,Thompson Raisins - Raisin,1 c,400,100g,0g,4g,0mg,40mg,92g,8g
3,"January 1, 2019",Breakfast,"Seeds, sunflower seed kernels, dry roasted, with salt added",1 cup,745,31g,64g,25g,0mg,838mg,3g,12g
4,"January 1, 2019",Dinner,Baked Cod - Baked Cod,4.8 oz,166,1g,5g,28g,59mg,552mg,0g,0g


,Date,Entry,Item,Amount,Calories,Carbs,Fat,Protein,Cholest,Sodium,Sugars,Fiber
459,"January 14, 2019",Dinner,Best Foods - Olive Oil Mayo,2 tbsp,120,1g,12g,0g,10mg,240mg,0g,0g
460,"January 14, 2019",Dinner,Thrive - Chicken Salad,0.5 cup,110,4g,1g,19g,40mg,45mg,0g,1g
461,"January 14, 2019",Dinner,Lettuce - Lettuce,2 cup shredded,10,2g,0g,1g,0mg,20mg,1g,1g
462,"January 14, 2019",Dinner,Generic - Bulk Almond Slices,0.13 c,85,3g,8g,3g,0mg,0mg,1g,2g
463,"January 14, 2019",Dinner,TreeTop - Applesauce no sugar added,1 cup,50,12g,0g,0g,0mg,0mg,11g,2g


# Save data to files

In [15]:
def dump_data(filename, df):
    """Write CSV file with dataframe contents. Return None."""
    df.to_csv(filename, index=False)
    return None

In [16]:
foodfile = output_dir + 'food_' + output_file
exercisefile = output_dir + 'exercise_' + output_file
notefile = output_dir + 'notes_' + output_file

dump_data(foodfile, food_log_flat)
dump_data(exercisefile, exercise_log_flat)
dump_data(notefile, notes_log_df)

In [17]:
# test output files by reloading
def load_data(filename):
    """Return dataframe loaded with pickled file."""
    return pd.read_csv(filename)

In [18]:
food_testdf = load_data(foodfile)
exercise_testdf = load_data(exercisefile)
note_testdf = load_data(notefile)

In [19]:
food_testdf.head()
exercise_testdf.head()
note_testdf.head()

,Date,Entry,Item,Amount,Calories,Carbs,Fat,Protein,Cholest,Sodium,Sugars,Fiber
0,"January 1, 2019",Breakfast,Baked Cod - Baked Cod,4.8 oz,166,1g,5g,28g,59mg,552mg,0g,0g
1,"January 1, 2019",Breakfast,Sweet Potato Soup,2 serving(s),638,103g,15g,32g,52mg,"3,436mg",40g,17g
2,"January 1, 2019",Breakfast,Thompson Raisins - Raisin,1 c,400,100g,0g,4g,0mg,40mg,92g,8g
3,"January 1, 2019",Breakfast,"Seeds, sunflower seed kernels, dry roasted, with salt added",1 cup,745,31g,64g,25g,0mg,838mg,3g,12g
4,"January 1, 2019",Dinner,Baked Cod - Baked Cod,4.8 oz,166,1g,5g,28g,59mg,552mg,0g,0g


,Date,Entry,Item,Amount,Calories,Minutes,Sets,Reps,Weight
0,"January 1, 2019",Cardiovascular,"Walking, 3.0 mph, mod. pace",walking dog,124,20,,NaN,NaN
1,"January 2, 2019",Strength Training,NaN,Bent Arm Barbell Pullover,,3,12,\n,NaN
2,"January 2, 2019",Strength Training,NaN,Leg Curls,,3,12,\n,NaN
3,"January 2, 2019",Strength Training,NaN,Leg Extension,,3,12,\n,NaN
4,"January 3, 2019",Cardiovascular,"Walking, 3.0 mph, mod. pace",walking dog,124,20,,NaN,NaN


,Note,Note Type
0,Starting Whole30 today.,Food Notes
1,"Also, did a lot of hauling today.",Exercise notes
2,Symptom - Knee Pain,Exercise notes
3,Symptom - Knee Pain,Exercise notes
